In [2]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier

import os
for dirname, _ , files in os.walk('Datasets'):
    for file in files:
        print(os.path.join(dirname, file))

Datasets\gender_submission.csv
Datasets\test.csv
Datasets\train.csv


In [3]:
train_df = pd.read_csv("Datasets/train.csv")
test_df = pd.read_csv('Datasets/test.csv')
gender_submission = pd.read_csv('Datasets/gender_submission.csv')

# Checking whether the subscription is accurate

This Kaggle notebook came with a sample subcription for this model, in which it predicts the survival rate by sex. Following the exercises in the kaggle course, I will check the degree of accuracy of the predictions for this subscription first before going into the model.

In [4]:
women_who_survived = train_df.loc[train_df.Sex == 'female']['Survived']
women_survival_rate = sum(women_who_survived) / len(women_who_survived)
print(f"Women Survival Rate %: {women_survival_rate * 100:.2f}")

men_who_survived = train_df.loc[train_df.Sex == 'male']['Survived']
men_survival_rate = sum(men_who_survived) / len(men_who_survived)
print(f"Women Survival Rate %: {men_survival_rate* 100:.2f}")


Women Survival Rate %: 74.20
Women Survival Rate %: 18.89


## Exploring Data

In this section I will be first exploring the datasets, in order to identify the presence of missing values, the features involved and how to interpret them.

First, taking a look at both train and test dataframes to check if they have the same amount of columns and their sizes.

In [5]:
train_rows, train_columns = len(train_df.index), len(train_df.columns)
test_size, test_columns = len(test_df.index), len(test_df.columns)
print(f"Test has {test_size} rows and {test_columns} columns, while train has {train_rows} rows and {train_columns} columns")

Test has 418 rows and 11 columns, while train has 891 rows and 12 columns


In [6]:
test_columns = test_df.columns.tolist()
train_columns = train_df.columns.tolist()
print(f"Train Columns:{sorted(train_columns)}\nTest Columns: {sorted(test_columns)}")

Train Columns:['Age', 'Cabin', 'Embarked', 'Fare', 'Name', 'Parch', 'PassengerId', 'Pclass', 'Sex', 'SibSp', 'Survived', 'Ticket']
Test Columns: ['Age', 'Cabin', 'Embarked', 'Fare', 'Name', 'Parch', 'PassengerId', 'Pclass', 'Sex', 'SibSp', 'Ticket']


As usual, test doesn't have the target for the model. Since we are going to check for missing values for both datasets at once, by combining them, I may have to drop the target for the train dataset for this treatment.

In [7]:
def append_data(df1: DataFrame, df2: DataFrame) -> DataFrame:
    return pd.concat(df1, df2)

NameError: name 'DataFrame' is not defined